In [1]:
import requests
import json
# r = requests.get("https://api.opendota.com/api/players/91886888/matches?region=2&limit=5")
# with open('matches_sri_5.json', 'w') as data_file:
#     json.dump(r.json(),data_file)

In [2]:
# import bigjson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as f:
#     j = bigjson.load(f)
#     element = j[4]
#     print(element)

# import ijson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as input_file:
#         # load json iteratively
#         parser = ijson.parse(input_file)
#         chat = ijson.items(input_file, 'features.item.chat.item')
#         for item in chat:
#             print('Chat: {}'.format(item))
# #         for prefix, event, value in parser:
# #             print('prefix={}, event={}, value={}'.format(prefix, event, value))

In [3]:
wins = {}
losses = {}
both = {}

# with open('matches_sri_5.json', 'r') as data_file:
with open('matches_sri_5.json', 'r') as data_file:
    data = json.load(data_file)
    
    for element in data:
        # print(element)
        r = requests.get("https://api.opendota.com/api/matches/" + str(element['match_id']))
        r = r.json()

        r.pop('all_word_counts', None)
        r.pop('match_id', None)
        r.pop('barracks_status_dire', None)
        r.pop('barracks_status_radiant', None)
#         r.pop('chat', None)
        r.pop('cluster', None)
        r.pop('cosmetics', None)
        r.pop('dire_score', None)
        r.pop('dire_team_id', None)
        r.pop('draft_timings', None)
        r.pop('duration', None)
        r.pop('engine', None)
#         r.pop('first_blood_time', None)
#         r.pop('game_mode', None)
        r.pop('human_players', None)
        r.pop('leagueid', None)
        r.pop('lobby_type', None)
        r.pop('match_seq_num', None)
        r.pop('negative_votes', None)
        r.pop('objectives', None)
        r.pop('picks_bans', None)
        r.pop('positive_votes', None)
        r.pop('radiant_gold_adv', None)
        r.pop('radiant_score', None)
        r.pop('radiant_team_id', None)
#         r.pop('radiant_win', None)
        r.pop('radiant_xp_adv', None)
        r.pop('skill', None)
        r.pop('start_time', None)
        r.pop('teamfights', None)
        r.pop('tower_status_dire', None)
        r.pop('tower_status_radiant', None)
        r.pop('version', None)
        r.pop('replay_salt', None)
        r.pop('series_id', None)
        r.pop('series_type', None)
        r.pop('players', None)
        r.pop('patch', None)
        r.pop('region', None)
        r.pop('throw', None)
        r.pop('loss', None)
        r.pop('win', None)
        r.pop('my_word_counts', None)
        r.pop('comeback', None)
        r.pop('stomp', None)
        r.pop('replay_url', None)
     
        pop_chat = []
        for index, item in enumerate(r['chat']):
            if item['slot'] == 10:
                pop_chat.append(index)

        print("pop index" + str(pop_chat))
        for item in reversed(pop_chat):
            print("Popped" + str(r['chat'].pop(int(item))))
        
        with open('chat_wheel.json', 'r',encoding='utf8') as data_file:
            chat_wheel = json.load(data_file)
            
        for idx, item in enumerate(r['chat']):
            if item['type'] == 'chatwheel':
#                 print(item['key'])
                item['type'] = 'chat'
                item['key'] = chat_wheel[item['key']].get('message')
        
        both[element['match_id']] = r
        print("Added to both.")
#         print(r)
        
        if element['radiant_win'] == True:
            wins[element['match_id']] = r
            print("Added to wins.")
        else:
            losses[element['match_id']] = r
            print("Added to losses.")
        with open('wins.json', 'w') as outfile:
            json.dump(wins, outfile)
        with open('losses.json', 'w') as outfile:
            json.dump(losses, outfile)
        with open('both.json', 'w') as outfile:
            json.dump(both, outfile)
            
# print("Wins:")
# print(wins)
# print("Losses:")
# print(losses)
# print("Both:")
# print(both)


pop index[0, 1]
Popped{'time': -226, 'type': 'chat', 'unit': 'SQUEE', 'key': 'tobis ded', 'slot': 10}
Popped{'time': -235, 'type': 'chat', 'unit': 'SQUEE', 'key': 'shhhh', 'slot': 10}
Added to both.
Added to wins.
pop index[]
Added to both.
Added to wins.
pop index[]
Added to both.
Added to losses.
pop index[]
Added to both.
Added to losses.
pop index[]
Added to both.
Added to wins.


In [4]:
wins

{5065183743: {'chat': [{'time': -82,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 9,
    'player_slot': 132},
   {'time': -62,
    'type': 'chat',
    'key': 'I know I can trust you guys.',
    'slot': 5,
    'player_slot': 128},
   {'time': -4,
    'type': 'chat',
    'key': '%s1 is missing!',
    'slot': 3,
    'player_slot': 3},
   {'time': -4,
    'type': 'chat',
    'key': '%s1 is missing!',
    'slot': 3,
    'player_slot': 3},
   {'time': 155,
    'type': 'chat',
    'key': '%s1 is missing!',
    'slot': 0,
    'player_slot': 0},
   {'time': 155,
    'type': 'chat',
    'key': '%s1 is missing!',
    'slot': 0,
    'player_slot': 0},
   {'time': 204,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 2,
    'player_slot': 2},
   {'time': 277,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 9,
    'player_slot': 132},
   {'time': 332,
    'type': 'chat',
    'key': '%s1 is missing!',
    'slot': 0,
    'player_slot': 0},
   {'time': 332,
    'type'

In [5]:
import copy
win_chats={}
lose_chats={}
for game in wins:
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']>4)]
    #print(game_copy)
    win_chats[game]=game_copy
    
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    lose_chats[game]=game_copy

for game in losses:
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']>4)]
    lose_chats[game]=game_copy
    
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    win_chats[game]=game_copy

print(win_chats) 
print(len(win_chats))
print(lose_chats) 
len(lose_chats)


{5065183743: {'chat': [{'time': -4, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 3, 'player_slot': 3}, {'time': -4, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 3, 'player_slot': 3}, {'time': 155, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 0, 'player_slot': 0}, {'time': 155, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 0, 'player_slot': 0}, {'time': 204, 'type': 'chat', 'key': 'Well played!', 'slot': 2, 'player_slot': 2}, {'time': 332, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 0, 'player_slot': 0}, {'time': 332, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 0, 'player_slot': 0}, {'time': 365, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 4, 'player_slot': 4}, {'time': 387, 'type': 'chat', 'key': 'Missing top!', 'slot': 2, 'player_slot': 2}, {'time': 487, 'type': 'chat', 'key': 'Well played!', 'slot': 2, 'player_slot': 2}, {'time': 498, 'type': 'chat', 'key': 'Well played!', 'slot': 3, 'player_slot': 3}, {'time': 507, 'type': 'chat', 'key': '%s1 is 

5

In [6]:
wordDict = {}
for game in both.keys():
    for i in both[game]['chat']:
        for word in i['key'].split():
            if word in wordDict:
                wordDict[word] = wordDict[word]+1
            else:
                wordDict[word]=1

In [7]:
for i in both.keys():
    print(i)

5065183743
5065165158
5065133302
5065105226
5054856538


In [8]:
wordDict

{'Well': 21,
 'played!': 21,
 'I': 9,
 'know': 1,
 'can': 1,
 'trust': 1,
 'you': 1,
 'guys.': 1,
 '%s1': 37,
 'is': 45,
 'missing!': 37,
 'Missing': 1,
 'top!': 1,
 'Get': 2,
 'Back!': 2,
 'Game': 7,
 'hard': 7,
 'Good': 11,
 'game,': 10,
 'well': 10,
 'played': 10,
 'GG': 1,
 'WP': 1,
 'gg': 11,
 'ggwp': 1,
 'Yahoo': 2,
 'On': 2,
 'my': 3,
 'way': 2,
 'ty': 1,
 '..': 1,
 'i': 2,
 'did': 1,
 'it': 1,
 'to': 3,
 'cross': 1,
 'ES': 1,
 'wall': 1,
 'luck,': 1,
 'have': 2,
 'fun': 1,
 'Affirmative': 8,
 'Ha': 2,
 'ha': 16,
 'ha!': 2,
 'like': 1,
 'what': 1,
 'see.': 1,
 'Thanks!': 2,
 'My': 4,
 'bad': 3,
 "Don't": 2,
 'give': 2,
 'up!': 2,
 'Space': 1,
 'created': 1,
 'end': 1,
 'Yes.': 1,
 'Yes!': 1,
 'purpose': 1,
 'fulfilled.': 1,
 'Careful!': 1,
 'ez': 1,
 'Your': 1,
 'future': 1,
 'grim.': 1,
 'noticed': 1,
 'a': 5,
 'child': 1,
 'in': 1,
 'horse': 1,
 'trough': 2,
 'with': 3,
 'metal': 1,
 'fencing': 1,
 'over': 1,
 'the': 2,
 'top': 1,
 'of': 2,
 'secured': 1,
 'zip': 1,
 'ties,': 

In [9]:
import operator
wordDictCounts = sorted(wordDict.items(), key=operator.itemgetter(1))

In [10]:
wordDictCounts[::-1]

[('is', 45),
 ('missing!', 37),
 ('%s1', 37),
 ('played!', 21),
 ('Well', 21),
 ('ha', 16),
 ('gg', 11),
 ('Good', 11),
 ('played', 10),
 ('well', 10),
 ('game,', 10),
 ('I', 9),
 ('Affirmative', 8),
 ('hard', 7),
 ('Game', 7),
 ('a', 5),
 ('My', 4),
 ('with', 3),
 ('bad', 3),
 ('to', 3),
 ('my', 3),
 ('me', 2),
 ('little', 2),
 ('Ben', 2),
 ('want', 2),
 ('of', 2),
 ('the', 2),
 ('trough', 2),
 ('up!', 2),
 ('give', 2),
 ("Don't", 2),
 ('Thanks!', 2),
 ('ha!', 2),
 ('Ha', 2),
 ('have', 2),
 ('i', 2),
 ('way', 2),
 ('On', 2),
 ('Yahoo', 2),
 ('Back!', 2),
 ('Get', 2),
 ('gege', 1),
 ('wp', 1),
 (':3c', 1),
 ('deal', 1),
 ('package', 1),
 ('water', 1),
 ('bath', 1),
 ('bonus', 1),
 ('nut', 1),
 ('HEROOOOOO', 1),
 ('A', 1),
 ('NEED', 1),
 ('s', 1),
 ('naughty', 1),
 ('being', 1),
 ('for', 1),
 ('slap', 1),
 ('and', 1),
 ('around', 1),
 ('boss', 1),
 ('daddy', 1),
 ('kitten.', 1),
 ('cum', 1),
 ('sexy', 1),
 ("Shapiro's", 1),
 ('be', 1),
 ('toes', 1),
 ('cute', 1),
 ('owo', 1),
 ('pics?',

In [11]:
words = list(wordDict.keys())

In [12]:
words

['Well',
 'played!',
 'I',
 'know',
 'can',
 'trust',
 'you',
 'guys.',
 '%s1',
 'is',
 'missing!',
 'Missing',
 'top!',
 'Get',
 'Back!',
 'Game',
 'hard',
 'Good',
 'game,',
 'well',
 'played',
 'GG',
 'WP',
 'gg',
 'ggwp',
 'Yahoo',
 'On',
 'my',
 'way',
 'ty',
 '..',
 'i',
 'did',
 'it',
 'to',
 'cross',
 'ES',
 'wall',
 'luck,',
 'have',
 'fun',
 'Affirmative',
 'Ha',
 'ha',
 'ha!',
 'like',
 'what',
 'see.',
 'Thanks!',
 'My',
 'bad',
 "Don't",
 'give',
 'up!',
 'Space',
 'created',
 'end',
 'Yes.',
 'Yes!',
 'purpose',
 'fulfilled.',
 'Careful!',
 'ez',
 'Your',
 'future',
 'grim.',
 'noticed',
 'a',
 'child',
 'in',
 'horse',
 'trough',
 'with',
 'metal',
 'fencing',
 'over',
 'the',
 'top',
 'of',
 'secured',
 'zip',
 'ties,',
 'As',
 'kid',
 'wouwd',
 'eat',
 'whowe',
 'bag',
 'Dowitos',
 'without',
 'swawwowing,',
 'h-h-hewwo??',
 'uwu',
 'think',
 'u',
 'might',
 'mixed',
 'some',
 'paint',
 'into',
 'those',
 'doritos',
 'too',
 'by',
 'accident',
 'buy',
 'feet',
 'pics?'

In [13]:
words.insert(0,'Radiant Wins')

In [14]:
import numpy as np
AllWords = np.zeros((len(both)*2,len(wordDict)+1), dtype=int)
game_index = 0
for game in win_chats.keys():
    AllWords[game_index][0]=1
    for line in win_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1
for game in lose_chats.keys():
    for line in lose_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1

In [15]:
AllWords

array([[ 1,  5,  5, ...,  0,  0,  0],
       [ 1,  3,  3, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  1,  1,  1],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0, 10, 10, ...,  0,  0,  0],
       [ 0,  1,  1, ...,  0,  0,  0]])

In [16]:
import pandas as pd 
df = pd.DataFrame(AllWords, columns = words) 

In [17]:
df

,Radiant Wins,Well,played!,I,know,can,trust,you,guys.,%s1,...,HEROOOOOO,nut,bonus,bath,water,package,deal,:3c,wp,gege
0,1,5,5,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0
1,1,3,3,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,1
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,2,2,2,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,5,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
8,0,10,10,2,0,0,0,0,0,11,...,0,0,0,0,0,0,0,0,0,0
9,0,1,1,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [20]:
y_test

array([1, 1])

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [23]:
y_pred = classifier.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[0 0]
 [2 0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
